## Modeling
- After some research I decided to turn the forecasting problem into a supervised classification problem as I have multivariate Data with a Target. 
- In a first step the dataframe was cleaned and transformed. Datapoints for recovering were removed
- In this notebook the data will be aggregated and the target shifted to the aggregated block before the minute of the failure
- First aggregation will be over 10 minutes
- All Sensors will be used

## Future Work
- Test more timeframes for aggregation
- PCA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('Solarize_Light2')

In [2]:
df = pd.read_csv('wp_clean.csv')

In [3]:
df.timestamp = pd.to_datetime(df.timestamp)

In [4]:
df = df.set_index('timestamp')

In [5]:
df.head()

,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,...,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_51,broken
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-04-01 00:00:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,...,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,201.3889,0
2018-04-01 00:01:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,...,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,201.3889,0
2018-04-01 00:02:00,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,15.01013,...,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,203.7037,0
2018-04-01 00:03:00,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,15.08247,...,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,203.1250,0
2018-04-01 00:04:00,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,15.08247,...,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,201.3889,0


In [6]:
df[df.broken == 1].index.tolist()

[Timestamp('2018-04-12 21:55:00'),
 Timestamp('2018-04-18 00:30:00'),
 Timestamp('2018-05-19 03:18:00'),
 Timestamp('2018-05-25 00:30:00'),
 Timestamp('2018-06-28 22:00:00'),
 Timestamp('2018-07-08 00:11:00'),
 Timestamp('2018-07-25 14:00:00')]

In [7]:
# get new dataframes that end with pump failure and aggregate over 10 timeframes

In [8]:
first = df.loc['2018-04-01 00:00:00':'2018-04-12 21:54:00']
first = first.resample('10min', origin=first.index[-1]).mean()
first.iloc[-1,-1] = 1

In [9]:
second = df.loc['2018-04-12 21:56:00':'2018-04-18 00:29:00']
second = second.resample('10min', origin=second.index[-1]).mean()
second.iloc[-1,-1] = 1

In [10]:
third = df.loc['2018-04-18 00:31:00':'2018-05-19 03:17:00']
third = third.resample('10min', origin=third.index[-1]).mean()
third.iloc[-1,-1] = 1

In [11]:
fourth = df.loc['2018-05-19 03:19:00':'2018-05-25 00:29:00']
fourth = fourth.resample('10min', origin=fourth.index[-1]).mean()
fourth.iloc[-1,-1] = 1

In [12]:
fifth = df.loc['2018-05-25 00:31:00':'2018-06-28 21:59:00']
fifth = fifth.resample('10min', origin=fifth.index[-1]).mean()
fifth.iloc[-1,-1] = 1

In [13]:
sixth = df.loc['2018-06-28 22:01:00':'2018-07-08 00:10:00']
sixth = sixth.resample('10min', origin=sixth.index[-1]).mean()
sixth.iloc[-1,-1] = 1

In [14]:
seventh = df.loc['2018-07-08 00:11:01':'2018-07-25 13:59:00']
seventh = seventh.resample('10min', origin=seventh.index[-1]).mean()
seventh.iloc[-1,-1] = 1

In [15]:
# Create new dataframe with all seven incidents
data = pd.concat([first,second, third, fourth, fifth, sixth, seventh], ignore_index=True)

In [16]:
data.head()

,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,...,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_51,broken
0,2.458999,47.157115,53.200950,46.354164,633.940975,75.863615,13.366248,16.136790,15.612343,15.049915,...,31.705729,41.601557,39.424188,65.393518,51.070605,38.194441,156.97335,67.346648,202.401625,0
1,2.453195,47.478296,53.164060,46.397568,635.393490,75.879854,13.360821,16.168255,15.761719,15.106337,...,31.484375,42.265622,38.975694,58.043980,51.128472,39.091435,181.85764,69.733797,201.215260,0
2,2.454080,48.723958,53.190103,46.319442,635.648140,78.037966,13.353586,16.177660,15.669850,15.084637,...,31.874998,40.572914,41.984954,52.777776,46.064816,39.872686,221.87500,78.240739,200.578710,0
3,2.454178,48.554686,53.120658,46.245659,636.238420,74.929848,13.341288,16.153065,15.792101,15.070170,...,32.343747,39.583330,37.586804,47.829862,47.337964,40.538193,202.40162,100.781266,216.869210,0
4,2.454867,48.949651,53.116316,46.176213,633.750000,72.600750,13.342011,16.135703,15.661170,15.085359,...,32.421872,39.062497,36.516201,46.006944,44.155093,39.178240,220.77546,83.159725,221.527780,0


In [17]:
# Check Dataframe
data.broken.value_counts()

0    15205
1        7
Name: broken, dtype: int64

In [20]:
# Save new Dataframe
# data.to_csv('transform_one.csv', sep=',', encoding='utf-8', index=False)